## Part I: Preprocessing
In this part, we will crop images, make masks and split them into 676 (26 x 26) boxes. Start by importing functions. 
Set info of the images for analysis. Sections to change are labelled in comments

In [ ]:
%load_ext autoreload
%autoreload 2

In [ ]:
import f_preprocessing 
import f_modelDetection
import f_coordFinder
import f_intensityMeasurement
import f_distributions
import f_ND6GastruloidScoring

wt = "WT"
mutant = "ND6" # change this to your mutant type 
conditions = [wt, mutant]

repeats = [1,2,3]

markers = ["DAPI", "SOX2", "BRA", "GATA3"] # change this to your markers 

channel_folders = { # change this to your markers and how theyre arranged on the CZI channels
        0: 'DAPI',
        1: 'SOX2',
        2: 'GATA3',
        3: 'BRA'
    }

directory = "CHIP_REPEATS_NEW" # directory as shown above
cropping_csv_path = "cropping.csv" # where all cropping info will be saved


Set where the czi files are saved in and other file directories. Arrange your CZI files in this format:

```
CHIP_REPEATS
├── 1 
│   ├── WT.czi
│   └── ND6.czi
├── 2
│   ├── WT.czi
│   └── ND6.czi
├── 3
│   ├── WT.czi
│   └── ND6.czi
```

Where the numbers represent your repeats and ND6 us what you set mutant as above 

### If Masks were made before cropping:
If you made the masks using the Raw image from the microscope then follow these steps.
1. Make sure they are named and inside the directory set above like this:
```
    CHIP_REPEATS
    ├── 1 
    │   ├── {condition}_{marker}_mask.tiff
    │   ├── WT_DAPI_mask.tiff
    │   ├── ND6_DAPI_mask.tiff
    │   ├── WT_SOX2_mask.tiff
    │   ├── ND6_SOX2_mask.tiff
    │   └── ...
    ├── 2
    │   ├── ...
    │   └── ...
    ├── 3
    │   ├──...
    │   └── ...
```
2. Convert CZI image into a tiff with the channels still intact using the block below

In [ ]:
f_preprocessing.convert_czi_to_tiff(directory)

After the step above step, you should use ImageJ/ Fiji to do these steps:
1. Manually crop and adjust angles 
2. Save coordinates for cropping in directory. (shown below)
2. Scale the cropped image to 7800 x 7800 pixels
3. Save the cropped scaled image in directory like this:

```
    CHIP_REPEATS
    ├── 1 
    │   ├── {condition}_scaled.tiff 
    │   ├── {condition}_coords.csv 
    │   ├── WT_scaled.tiff
    │   ├── ND6_scaled.tiff
    │   ├── WT_coords.csv
    │   └── ND6_coords.csv
    ├── 2
    │   ├── ...
    │   └── ...
    ├── 3
    │   ├──...
    │   └── ...
```

A tutorial of how to do the above steps right here: (link). 

Once youre done, Run the block below which:
1. Split the Tiffs into channels according to the channel folders set above 
2. Compile cropping coordinates into a cropping.csv file. 

After running the block below, if the raw images were rotated in ImageJ (before cropping), the angle for adjustment needs to be added to the csv file (default is 0).

In [ ]:
# Check the dimensions of your tiff files
# f_preprocessing.check_dimensions(directory) # it should show your scaled images have dimensions ({number of channels}, 7800, 7800)
f_preprocessing.check_dimensions("CHIP_REPEATS_NEW/2") # it should show your scaled images have dimensions ({number of channels}, 7800, 7800)

In [ ]:
# split Tiffs into channels 
f_preprocessing.split_tiff_into_channels(directory, repeats, conditions, channel_folders)

# Compile cropping coordinates (to crop masks according to how images were cropped)
f_preprocessing.compile_crop_coordinates(directory, repeats, conditions, cropping_csv_path)

Crop and resize masks. They will replace the original mask tiff. Then we check if all tiffs we have in our directory are the same size.

In [ ]:
f_preprocessing.crop_masks(directory, cropping_csv_path, conditions, repeats, markers)
print("finished cropping all masks!")
f_preprocessing.check_dimensions(directory)

Now all Images and Masks are aligned and the same size, we will split them into 676 boxes using a 26 x 26 grid. This process might take awhile. (7-10min)

In [ ]:
f_modelDetection.make_grid_and_split_all(directory, markers, conditions, repeats)

Manually select which boxes to include in the analysis. When finishing, all coordinates and diameters of models will be saved and you can move on to Radius adjustment. Run the second block below to create another folder called "boxes_tiff_selected" that only include the boxes you have selected and have been assigned a different index. 

In [ ]:
model_marker = ["DAPI"]
f_modelDetection.detect_blob_all(directory, model_marker, conditions, repeats)

In [ ]:
f_modelDetection.boxes_tiff_selected(directory, repeats, conditions, markers)

## Part II: Radius Adjustment
In this part, we will use Repeat 1, DAPI to manually adjust and set the radius sizes for our Bins. Set values in the adjusting_values dictionary to save a folder of images in this format "{directory}/{repeat}/adjusting/{marker_adjusting}" of the marker along with a circle rperesenting the radius you set. DAPI is used to adjust for the Radius of the model. BRA is used to adjust for the middle radius, SOX2 is used to adjust for the smallest Radius. 

In [ ]:
radius_values = {"DAPI": 110, # change these numbers (theyre just an example)
                     "BRA": 73.923,
                     "SOX2": 51.789 }


Run the block below to start adjustment. Making binary masks will take ~1min. Then it will make images saved in your directory under a folder called adjusting for you to see what the radius you set above looks like on the images (3-4min). Coordinates found will be saved in a folder called 'coordinates' in your directory. 

adjusting=False means no images will be saved (makes it run faster). adjusting = True makes it save all images to adjust for radius (takes a long time but useful for adjusting radius or trying to visualize how the radius looks like on the image / how well coordinate detection is going)

In [ ]:
f_coordFinder.run_R2(directory, repeats, conditions, radius_values, adjusting=True)


## Part III: Bin Intensity Measurement
In this part, using the radius we have adjusted and set above, we will load the coordinates saved and measure the RAW intensity for each marker (yes including DAPI). These raw coordinates will be saved in an npz in a folder called 'intensities'. Before we do that, lets make a mask for GATA3 to remove noise as the image is really noisy! from using pink nailpolish >:(

In [ ]:
f_intensityMeasurement.make_GATA3_filter(directory, repeats, conditions)

In [ ]:
f_intensityMeasurement.intensities_per_marker(directory, repeats, conditions, markers, radius_values)

Now we will normalize all the markers we have by DAPI.

In [ ]:
f_intensityMeasurement.normalize_markers(directory, repeats, conditions, markers)

You might want to take a quick look at meta_intensities.csv to see if your values kind of make sense. If you're satisfied, its time to visualize the data! (plot it!). You can find all plots in your directory, in a folder called 'plots'.

In [ ]:
meta_path = f'{directory}/meta_intensities.csv'


for repeat in repeats:
    save_dir = f'{directory}/{repeat}/plots'
    f_intensityMeasurement.bar_plot(meta_path, repeat, plot_type='line', save_dir=save_dir)

for repeat in repeats:
    for marker in markers:
        if marker != "DAPI":
            save_dir = f'{directory}/{repeat}/plots'
            f_intensityMeasurement.plot_marker_condition_overlap(meta_path, repeat, marker, save_dir=save_dir)

## Part IV: Diameter and Intensity Distribution 
In this section, we will find the diameter (by DAPI) and Intensity (of each marker) of each gastruloid. Then we will plot this data on an overlap density plot (WT vs mutant). So the plots we will make are
1.  Gastruloid Diameter
2.  All markers Intensity 

We are trying to find a relationship between correct marker expression localisation so we will plot a scatter plot where the x-axis is the gastruloid diameter and the y-axis is the intensity of specific markers in specific regions. Here, we are using:
- SOX2: inner bin 
- BRA: middle bin
- GATA3: outer bin


In [ ]:
f_distributions.get_distributions(directory, repeats, conditions, markers)

In [ ]:
marker_loc_dict = {'SOX2': 'inner', 'BRA': 'mid', 'GATA3': 'outer'}

for repeat in repeats:
    for marker, _ in marker_loc_dict.items():
            repeats_data = [
                (directory, repeat, 'WT', f'{directory}/{repeat}/intensities/meta_individual_WT.csv', f'{directory}/{repeat}/distribution/WT_DAPI.csv', marker),
                (directory, repeat, 'ND6', f'{directory}/{repeat}/intensities/meta_individual_ND6.csv', f'{directory}/{repeat}/distribution/ND6_DAPI.csv', marker),
            ]

            # f_distributions.combined_plot_intensity_vs_diameter(
            #     repeats_data,
            #     marker_loc_dict
            # )
            
            f_distributions.combined_plot_intensity_vs_DAPIintensity(
                repeats_data,
                marker_loc_dict
            )

## Part V: Scoring ND6 Gastruloids and Plotting
In this section, we will score each ND6 gastruloid by comparing it to the WT of its repeat. The block below scores each gastruloid based on how similar their marker expression localisation is compared to WT. Higher score means less of a phenotype, lower score means more of a phenotype. You can see the results in your directory under this file name: "nd6_overlap_scores_{marker}.csv"

In [ ]:
f_ND6GastruloidScoring.score_gastruloid(directory, repeats, markers, conditions)

Run the code below to get the final score of each gastruloid (final is the average between BRA, SOX2 and GATA3 scores)

In [ ]:
f_ND6GastruloidScoring.final_score_gastruloid(directory, repeats, conditions)

Run the code below to get a figure of 5 gastruloids (either top scoring, middle socring or low scoring). The figure of each gastruloid is each of the channels seperately and one merged (excluding DAPI)

In [ ]:
# selection must be 'top', 'bottom', or 'middle'
f_ND6GastruloidScoring.plot_gastruloid_scoring(directory, repeats, "bottom", conditions)

Run the code below to get a Scatterplot where the x axis is gastruloid score and the y axis is DAPI Intensity. This is to see whether more cells (higher DAPI intensity) is related to higher scoring ND6 gastruloids. 

In [ ]:
f_ND6GastruloidScoring.DAPIIntensity_vs_score_scatterplot(directory, repeats, conditions)

In [ ]:
f_ND6GastruloidScoring.DAPIIntensity_vs_score_scatterplot_combined(directory, repeats, conditions)

Run the code below with a specific gastruloid ID (and their condition and repeat) to save a figure of that gastruloid that contains the channels seperate and merged (excluding DAPI)

In [ ]:
idx = 587
condition = 'WT'
repeat = 3
f_ND6GastruloidScoring.channels_plot_any(idx, directory, repeat, condition)

Run the code below to get the average DAPI Intensity in each repeat and condition. Will be saved in directory in a csv named "meta_DAPIintensity.csv"

In [ ]:
f_ND6GastruloidScoring.DAPI_average_intensity(directory, repeats, conditions)